In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init

import torchvision.utils as v_utils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
epoch = 50
batch_size = 512
learning_rate = 0.0002
num_gpus = 1
z_size = 50
middle_size = 200

In [4]:
mnist_train = dset.MNIST("drive/MyDrive", train=True, transform=transforms.ToTensor(), target_transform=None, download=False)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [37]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    self.layer1 = nn.Sequential(OrderedDict([
                                             ('fc1', nn.Linear(z_size, middle_size)),
                                             ('bn1', nn.BatchNorm1d(middle_size)),
                                             ('act1', nn.ReLU())
    ]))
    self.layer2 = nn.Sequential(OrderedDict([
                                             ('fc2', nn.Linear(middle_size,784)),
                                             ('bn2', nn.BatchNorm1d(784)),
                                             ('tanh', nn.Tanh())
    ]))

  def forward(self,z):
    out = self.layer1(z)
    out = self.layer2(out)
    out = out.view(-1, 1,28,28)

    return out

In [38]:
from torchsummary import summary

In [41]:
gen = Generator().cuda()
print(gen)

Generator(
  (layer1): Sequential(
    (fc1): Linear(in_features=50, out_features=200, bias=True)
    (bn1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU()
  )
  (layer2): Sequential(
    (fc2): Linear(in_features=200, out_features=784, bias=True)
    (bn2): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (tanh): Tanh()
  )
)


In [56]:
a = torch.ones(512,50).cuda()
print(a.size())

torch.Size([512, 50])


In [57]:
b = gen.forward(a)
print(b.size())

torch.Size([512, 1, 28, 28])


In [67]:
summary(gen, (50), 512)

TypeError: ignored

In [6]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.layer1 = nn.Sequential(OrderedDict([
                                             ('fc1', nn.Linear(784,middle_size)),
                                            #  ('bn1', nn.BatchNorm1d(middle_size)),
                                             ('act1', nn.LeakyReLU())
    ]))
    self.layer2 = nn.Sequential(OrderedDict([
                                             ('fc2', nn.Linear(middle_size,1)),
                                             ('bn2', nn.BatchNorm1d(1)),
                                             ('act2', nn.Sigmoid())
    ]))

  def forward(self,x):
    out = x.view(batch_size//num_gpus, -1)
    out = self.layer1(out)
    out = self.layer2(out)

    return out

In [7]:
generator = nn.DataParallel(Generator()).cuda()
discriminator = nn.DataParallel(Discriminator()).cuda()

In [8]:
gen_params = generator.state_dict().keys()
dic_params = discriminator.state_dict().keys()

for i in gen_params:
  print(i)

module.layer1.fc1.weight
module.layer1.fc1.bias
module.layer1.bn1.weight
module.layer1.bn1.bias
module.layer1.bn1.running_mean
module.layer1.bn1.running_var
module.layer1.bn1.num_batches_tracked
module.layer2.fc2.weight
module.layer2.fc2.bias


In [9]:
loss_func = nn.MSELoss()
gen_optim = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5,0.999))
dis_optim = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5,0.999))

ones_label = torch.ones(batch_size,1).cuda()
zeros_label = torch.zeros(batch_size,1).cuda()

In [10]:
for i in range(epoch):
  for j,(image,label) in enumerate(train_loader):
    image = image.cuda()

    dis_optim.zero_grad()

    z = init.normal(torch.Tensor(batch_size, z_size), mean=0, std=0.1).cuda()
    gen_fake = generator.forward(z)
    dis_fake = discriminator.forward(gen_fake)

    dis_real = discriminator.forward(image)
    dis_loss = torch.sum(loss_func(dis_fake, zeros_label)) \
               + torch.sum(loss_func(dis_real, ones_label))
    dis_loss.backward(retain_graph=True)
    dis_optim.step()

    
    gen_optim.zero_grad()

    z = init.normal(torch.Tensor(batch_size, z_size), mean=0, std=0.1).cuda()

    gen_fake = generator.forward(z)
    dis_fake = discriminator.forward(gen_fake)

    gen_loss = torch.sum(loss_func(dis_fake, ones_label))

    gen_loss.backward()
    gen_optim.step()


    if j % 100 == 0:
      print(gen_loss, dis_loss)
      torch.save([generator,discriminator], './model/vanilla_gan.pkl')

      print("{}th ineration gen_loss: {} dis_loss: {}".format(i, gen_loss.data, dis_loss.data))
      v_utils.save_image(gen_fake.data[0:25], "./result/gen_{}_{}.png".format(i,j), nrow=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


tensor(0.2944, device='cuda:0', grad_fn=<SumBackward0>) tensor(0.5873, device='cuda:0', grad_fn=<AddBackward0>)


FileNotFoundError: ignored